<a href="https://colab.research.google.com/github/djili/data-processing/blob/main/movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lire un fichier CSV contenant des informations sur des films(nous utiliserons le RDD du dataframe df.rdd).
Utiliser des accumulateurs pour suivre le nombre total de lignes et le nombre de lignes valides.
Utiliser des variables broadcast pour distribuer une liste de genres de films à surveiller.
Calculer le nombre total de films pour chaque genre surveillé.
Identifier les années avec le plus grand nombre de films pour chaque genre surveillé.

In [1]:
from pyspark.sql import SparkSession
import json

In [2]:
# Créer une session Spar
spark = SparkSession.builder.appName("AnalyseFilms").getOrCreate()

In [3]:
# Lire le fichier CSV avec en-têtes
df = spark.read.option("header", True).csv("/content/sample_data/movies_metadata.csv")

In [4]:
#Récupérer le RDD depuis le DataFrame
rdd = df.rdd

In [5]:
# Déclarer les accumulateurs pour compter les lignes
total_lines = spark.sparkContext.accumulator(0)
valid_lines = spark.sparkContext.accumulator(0)

In [6]:
# Définir les genres à surveiller et les diffuser avec broadcast
genres_surveilles = ["Drama", "Comedy", "Action", "Thriller"]
broadcast_genres = spark.sparkContext.broadcast(genres_surveilles)

In [7]:
def traiter_ligne(row):
    global total_lines, valid_lines
    total_lines += 1

    try:
        genres_col = row['genres']
        release_date = row['release_date']

        if not genres_col or not release_date:
            return []

        # Extraire l'année
        year = release_date[:4]

        # Nettoyage et parsing du champ genres
        genres_json = json.loads(genres_col.replace("'", "\""))
        genres_extraits = [g['name'] for g in genres_json if 'name' in g]

        # Filtrer selon les genres surveillés
        genres_valides = [g for g in genres_extraits if g in broadcast_genres.value]

        if genres_valides and year.isdigit():
            valid_lines += 1
            return [(g, year) for g in genres_valides]
        else:
            return []
    except:
        return []


In [8]:
# Appliquer la fonction sur le RDD
genre_annee_rdd = rdd.flatMap(traiter_ligne)

In [9]:
# Compter le nombre total de films par genre surveillé
genre_count = genre_annee_rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b)

In [10]:
# Compter le nombre de films par (genre, année)
genre_year_count = genre_annee_rdd.map(lambda x: ((x[0], x[1]), 1)).reduceByKey(lambda a, b: a + b)

In [11]:
# Trouver l'année avec le plus grand nombre de films pour chaque genre
max_year_by_genre = genre_year_count.map(lambda x: (x[0][0], (x[0][1], x[1]))).groupByKey().mapValues(lambda values: max(values, key=lambda v: v[1]))

In [12]:
# Afficher les résultats
print("Nombre total de lignes :", total_lines.value)
print("Nombre de lignes valides :", valid_lines.value)

print("\n🎬 Nombre total de films par genre surveillé :")
for genre, count in genre_count.collect():
    print(f"{genre} : {count}")

print("\n📅 Année avec le plus de films pour chaque genre surveillé :")
for genre, (year, count) in max_year_by_genre.collect():
    print(f"{genre} : {year} ({count} films)")

Nombre total de lignes : 0
Nombre de lignes valides : 0

🎬 Nombre total de films par genre surveillé :
Comedy : 11943
Action : 6046
Drama : 18669
Thriller : 7139

📅 Année avec le plus de films pour chaque genre surveillé :
Comedy : 2014 (506 films)
Action : 2011 (232 films)
Drama : 2014 (752 films)
Thriller : 2015 (354 films)
